# Create a `Mask` Reference File using the `RFP`

This notebook will guide users through two different scenarios when generating the `Mask` reference file using the RFP. These examples are: 

1. A user has a data quality flag (DQ) array generated from their own scripts and methods and would like to create a CRDS Mask Reference file with the DQ array. 
   
2. A user has a list of files and would like to generate a CRDS Mask Reference file using the RFP `Mask` module's algorithms.

-----

## Setup

In order to run this notebook and generate the Mask reference file, a `conda` environment must be created that has the `roman-wfi-reference-file-pipeline` code installed. 

To download `conda` on your machine, please follow the installation instructions on the [`conda` website](https://docs.conda.io/projects/conda/en/latest/user-guide/install/index.html). 

Once `conda` is installed, go to [the RFP's Github page](https://github.com/spacetelescope/roman-wfi-reference-pipeline) and follow [the RFP installation instructions](https://github.com/spacetelescope/roman-wfi-reference-pipeline#installation).

When the code is properly installed, users can run the notebook examples.

-----

## Imports

A few packages are needed when running this notebook. We will import them in the cell below:

In [ ]:
# Used to generate the reference file (required!)
from wfi_reference_pipeline.resources.make_dev_meta import MakeDevMeta
from wfi_reference_pipeline.reference_types.mask.mask import Mask

# Used to get the current bitvals from roman_datamodels
from roman_datamodels.dqflags import pixel as dq

# Used to easily retrieve a filelist
import glob
import os

# Used to plot and create fake data for this notebook, optional
import numpy as np
import matplotlib.pyplot as plt
import asdf
from astropy.io import fits

## About the RFP `Mask` Module

The `Mask` class inherits from the `ReferenceType` base class. User required inputs for the `Mask` object are:

1. Metadata: This will be in the CRDS meta information and it contains information about the file delivery such as reference file type, delivery date, description, author, instrument name and detector, etc.
   
2. Either a valid DQ array or a filelist

Users create a meta object using `MakeDevMeta` to store the metadata (from 1.) and input this object and their array/filelist (from 2.) to create a `Mask` object. Once the `Mask` object is created, users can run different commands (such as `make_mask_image()` if a filelist was inputted), or generate the CRDS-ready reference file using `generate_outfile()`. 

-----

## Example 1: Creating a Mask Reference File with a Pre-computed DQ Array

In this example, create a Mask reference file for `WFI05`. We will begin by creating a fake DQ array in the cell below.

In this fake mask, a user found an electronic effect from the first two amplifiers of `WFI05`. They are masking these columns from their science analysis by using the flag `DO_NOT_USE` on these columns. Additionally, they found that the upper right corner of the detector had bad reference pixels, so they mark the top 100 rows of the detector to have the flag `BAD_REF_PIXEL`.

We will also add some of the following flags in random positions: 

- `HOT` pixels
- `LOW_QE` pixels

Since a pixel can have multiple flags, we will use bitwise addition when adding flags. Note that the mask must have a shape of `4096x4096`.

In [ ]:
# Creating an empty mask to populate
fake_mask = np.zeros((4096, 4096), dtype=np.uint32)

# Adding the DO_NOT_USE flag to first two columns
fake_mask[:, :2] += dq["DO_NOT_USE"].value

# Adding the BAD_REF_PIXEL flags to top 100 rows
fake_mask[-100:, :] += dq["BAD_REF_PIXEL"].value

# Choosing 250 random pixel indices to have the HOT flag
npoints = 250
idx_hot = np.random.choice(fake_mask.size, size=npoints, replace=False)

fake_mask.flat[idx_hot] += dq["HOT"].value

# Choosing 250 random pixel indices to have the LOW_QE flag
npoints = 500
idx_lowqe = np.random.choice(fake_mask.size, size=npoints, replace=False)

fake_mask.flat[idx_lowqe] += dq["LOW_QE"].value

Now that we have a fake DQ array, we will need to create a `MakeDevMeta` object to store the metadata used in the CRDS submission. A full list of the metadata options for `Mask` are printed below.

In [ ]:
tmp = MakeDevMeta(ref_type="MASK")

print("The default metadata values are: ", tmp.meta_mask)

# Manually setting various metadata
tmp.meta_mask.use_after = '2020-05-01T00:00:00.000'
tmp.meta_mask.author = "A nice user"
tmp.meta_mask.instrument_detector = "WFI05"
tmp.meta_mask.description = "Testing creation of a Mask reference file with an inputted DQ array."

print("The new metadata values are: ", tmp.meta_mask)

Now we can create the `Mask` object. For this example, we will set meta_data to `tmp.meta_mask` (the metadata we just altered). We will set `file_list` to `None`, since we already have the DQ array we want to input. We will set `ref_type_data` to the DQ array (`fake_mask`) that the user made above. We also will specify a output filename for the reference file (we will use `mask_from_dq_arr.asdf`) and we will set `clobber` to `True` so we can overwrite existing files. 

Once we successfully create the `Mask` object, we can generate the reference file using the function `generate_outfile()`.

In [ ]:
# Creating the Mask object
rfp_mask = Mask(meta_data=tmp.meta_mask,
                file_list=None,
                ref_type_data=fake_mask,
                outfile="mask_from_dq_arr.asdf",
                clobber=True)

# Generating the CRDS-compliant reference file
rfp_mask.generate_outfile()

Congrats! A Mask reference file has been successfully created using an existing DQ array! We can plot it below (zooming in on two different portions of the detector). We will check if the upper 100 rows have been marked as `BAD_REF_PIXEL`.

In [ ]:
with asdf.open(rfp_mask.outfile) as af:

    mask = af["roman"]["dq"]

    # Plotting the DO_NOT_USE pixels
    fig = plt.figure(figsize=(10,10))

    # Creating a mask for the BAD_REF_PIXEL bitval
    mask_for_BRP = (mask & dq.BAD_REF_PIXEL.value) != 0

    plt.imshow(mask_for_BRP,
               cmap='viridis',
               origin="lower")

    # Since the detector is so large, we will zoom in at top
    plt.ylim(3000, 4096)

    # Adding labels and title
    plt.xlabel("X (pix)")
    plt.ylabel("Y (pix)")
    plt.title("BAD_REF_PIXEL Locations on WFI05")

    plt.colorbar()

    plt.show()

-----
## Example 2: Creating a Mask Reference File using a Filelist

In this second example, we will use the Mask modules `make_mask_image()` function to identify bad pixels using a list of flat files. We will also show users how to alter the values used in the bad pixel algorithms.

First we will get a list of files from the `OTP00615_SmoothDarkptA_TV2a_R1_MCEB` dataset for `WFI09`. 

In [ ]:
flat_filelist = glob.glob("/grp/roman/GROUND_TESTS/TVAC2_IRRCcorr/ASDF/NOM_OPS/OTP00615_SmoothDarkptA_TV2a_R1_MCEB/**/*WFI09*")[:2]

print(f"List of files to be analyzed:\n")
print("\n".join(str(os.path.basename(file)) for file in flat_filelist))

Similar to the first example, we will create a `MakeDevMeta` object and set some metadata that will be used in the CRDS submission.

In [ ]:
tmp = MakeDevMeta(ref_type="MASK")

print("The default metadata values are: ", tmp.meta_mask)

# Manually setting various metadata
tmp.meta_mask.use_after = '2020-05-01T00:00:00.000'
tmp.meta_mask.author = "Another very nice user"
tmp.meta_mask.instrument_detector = "WFI09"
tmp.meta_mask.description = "Testing creation of a Mask reference file with a filelist."

print("The new metadata values are: ", tmp.meta_mask)

Now we will create the `Mask` object. We will set `file_list` to the list of flat files, set `ref_type_data` to `None` since we have no existing DQ array. We will also set the `outfile` to be `mask_from_filelist.asdf`.

In [ ]:
# Creating the Mask object
rfp_mask = Mask(meta_data=tmp.meta_mask,
                file_list=flat_filelist,
                ref_type_data=None,
                outfile="mask_from_filelist.asdf",
                clobber=True)

To run the bad pixel algorithms, we will use the function make_mask_image(). Our user doesn't want the data to be smoothed when identifying bad pixels from flats so they will set `from_smoothed` to `False`, and they also want the maximum `LOW_QE` value to be higher than the default for their science, so they will change `max_low_qe_signal` from `0.5` to `0.55`. They also will want to save the normalized image that is created to inspect, so they will set `normalized_path` to the path where they would like to save it. 

In [ ]:
# TODO: Figure out the validation issues... should not be in final notebook
os.environ["ROMAN_VALIDATE"] = "false"

rfp_mask.make_mask_image(from_smoothed=False,
                         max_low_qe_signal=0.55,
                         normalized_path="./")

In [ ]:
# Generating the CRDS-compliant reference file
rfp_mask.generate_outfile()

Congrats! You created a CRDS ready bad pixel mask from a list of files. Let's inspect the output.

We will plot the normalized slope image that was created:

In [ ]:
normalized_image = fits.getdata("normalized_image.fits")

# Plotting the normalized image
fig = plt.figure(figsize=(10,10))

plt.imshow(normalized_image,
           origin="lower",
           vmin=0)

plt.title("Normalized Slope Image of WFI09")
plt.colorbar()

plt.show()

We will also print out the percentages of pixels flagged. We can also plot the overall mask; only dead pixels will be highlighted but users should feel free to change to any other flag of their choosing.

Note that at the time of this notebook there are no explicit flags for OPEN/ADJ. `RESERVED_5` is used for adjacent pixels and `RESERVED_6` is used for open pixels.

In [ ]:
flags_from_flats = ["DEAD", "LOW_QE", "RESERVED_5", "RESERVED_6"]

with asdf.open("mask_from_filelist.asdf") as af:

    # Extracting the mask
    mask = af["roman"]["dq"]

    # Printing the percentage of pixels flagged
    for flag in flags_from_flats: 

        filtered_mask = (mask & dq[flag].value) != 0
        percentage_pix = 100 * np.count_nonzero(filtered_mask) / mask.size

        print(f"The percentage of pixels flagged for {flag} are {percentage_pix}%")

    # Plotting the DEAD pixels
    filtered_mask = (mask & dq["DEAD"].value) != 0

    fig = plt.figure(figsize=(10,10))

    plt.imshow(filtered_mask,
               origin="lower")

    plt.title("DEAD pixels in WFI09")

    plt.show()

-----

This notebook was written by Sierra Gomez on October 10, 2025.